# Predicted Value Project for Cross Sell

Anna Hathaway
* Goals: predict if a customer is going to increase products in their insurance coverage

## Data Wrangling 

### Data Imports

In [ ]:
# imports
import pandas as pd
import numpy as np
pd.set_option('mode.use_inf_as_na', True)
from sklearn.model_selection import train_test_split
import matplotlib as plt
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.metrics import classification_report
import joblib
from sklearn.metrics import accuracy_score
from xgboost import cv
from dython.nominal import identify_nominal_columns
from dython.nominal import associations
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import roc_auc_score, roc_curve
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

In [ ]:
# importing renewal data
renewal_df = pd.read_pickle("c:/Users/H008906/OneDrive - Principal Financial Group/expected value/renewal_data.pkl")

In [ ]:
# selecting columns form renewal data 
renewal_df['RENEWAL_DATE'] = pd.to_datetime(renewal_df['RENEWAL_DATE'])
# renewal_df['COVERAGE_START_YEAR'] = renewal_df['COVERAGE_EFFECTIVE_DATE'].dt.year
# renewal_df['RENEWAL_YEAR'] = renewal_df['RENEWAL_DATE'].dt.year
# renewal_df['COVERAGE_DURATION'] = renewal_df['RENEWAL_YEAR'] - renewal_df['COVERAGE_START_YEAR']
renewal_df['RTN'] = renewal_df['RENEWED_CURRENT_PREMIUM'] / renewal_df['RENEWED_NEEDED_PREMIUM']
renewal_df['RTN'].fillna(0, inplace = True)

cols_to_use = ['CASE','PRODUCT','RENEWAL_DATE','ENTERING_CURRENT_PREMIUM','DENTAL','LIFE','LTD','STD','PRODUCTSCOUNT','RG','BROKER_STATUS','RTN']

cols_not_to_use = ['Unnamed: 0','SALES_OFFICE','OFFICE','ALLIANCE','BAND','BAND_GROUP','CURRENT_BAND','BAND_GROUP2','RATE_CHANGE_BAND','ENTERING_RTGT_COUNT',
'ENTERING_CALC_WT','ENTERING_CALC_PROFIT','RENEWED_RTGT_COUNT','RENEWED_CALC_WT','RENEWED_USED_WT','RENEWED_PREM_SPEND','RENEWED_CALC_PROFIT','LAPSED_CVG_COUNT',
'LAPSED_CURRENT_PREMIUM','LAPSED_LIVES','LAPSED_RTGT_COUNT','LAPSED_RTGT_PREMIUM','LAPSED_CALC_WT','LAPSED_NEEDED_PREMIUM','LAPSED_TARGET_PROFIT','LAPSED_CURRENT_PROFIT',
'LAPSED_CALC_PROFIT','NO_CHANGE','RENEWED_AMEND_WT','RENEWED_MPD_WT', 'ENTERING_CVG_COUNT','CASE_SIZE_GROUP','CASE_SIZE_GROUP2','CASE_SIZE_GRP3','RENEWED_CVG_COUNT',
'RENEWED_CURRENT_PREMIUM','RENEWED_LIVES','RENEWED_RTGT_PREMIUM','RENEWED_NEEDED_PREMIUM',
'RENEWED_TARGET_PROFIT','RENEWED_CURRENT_PROFIT', 'RENEWED_USED_PROFIT','RENEWED_RENEWAL_PREMIUM', 'PERSISTED','DURATION_GROUP','RENEWAL_MONTH','BROKER_PAYEE', 'BROKER', 
'ENTERING_TARGET_PROFIT','ENTERING_CURRENT_PROFIT','ENTERING_USED_PROFIT','ENTERING_LIVES','RENEWAL_YEAR','COVERAGE_START_YEAR','COVERAGE_EFFECTIVE_DATE']

renewal_df = renewal_df[cols_to_use]

In [ ]:
# group renewal data by case/renewal date then create target cross sell variable 
renewal_grouped = renewal_df.groupby(['CASE','RENEWAL_DATE'])['PRODUCT'].count().reset_index()
renewal_grouped = renewal_grouped.rename(columns={'PRODUCT':'product_count'})
renewal_grouped = renewal_grouped.sort_values(by=['CASE','RENEWAL_DATE'])
renewal_grouped['count_diff'] = renewal_grouped.groupby(['CASE'])['product_count'].diff()
renewal_grouped['cross_sell'] = renewal_grouped['count_diff'].apply(lambda x: 1 if x > 0 else 0)

# merge new grouped data with target variable data with renewal data
renewal = pd.merge(renewal_df, renewal_grouped, on=['CASE','RENEWAL_DATE'], how='inner')

# add new RG column
renewal['RG_binary'] = renewal['RG'].apply(lambda x: 1 if x=='RG' else 0)

# create dataset with one row per case
renewal_agg = renewal.groupby(['CASE']).aggregate(
    {'ENTERING_CURRENT_PREMIUM':'mean', 
     'DENTAL':'max', 'LIFE':'max', 'LTD':'max', 'STD':'max',
     'PRODUCTSCOUNT':'max',
     #'COVERAGE_DURATION':'mean',
     'cross_sell':'max',
     'RG_binary':'max',
     'RTN':'mean',
     }).reset_index()

# get the broker status for each case 
grouped_df=renewal.groupby('CASE')['BROKER_STATUS']
most_common_values=grouped_df.apply(lambda x: x.mode().iloc[0])
most_common_values = most_common_values.reset_index()
# need to make all statuses lowercase to match 
most_common_values['BROKER_STATUS'] = most_common_values['BROKER_STATUS'].apply(str.lower)
# combine silver and elite statuses together because they are correlated
most_common_values['BROKER_STATUS'] = np.where(most_common_values['BROKER_STATUS'] == 'silver', 'silver/elite', most_common_values['BROKER_STATUS'])
most_common_values['BROKER_STATUS'] = np.where(most_common_values['BROKER_STATUS'] == 'elite', 'silver/elite', most_common_values['BROKER_STATUS'])
# get the dummies for broker status and store it in a variable then change to 0 1
broker_dummies = pd.get_dummies(most_common_values.BROKER_STATUS)
broker_dummies = broker_dummies.astype(int)
# Concatenate the dummies to original dataframe
most_common_values_dum = pd.concat([most_common_values, broker_dummies], axis=1)
# drop the other status column 
most_common_values_dum = most_common_values_dum.drop(['BROKER_STATUS', 'not available', 'missing info', 'multiple statuses', 'gold', 'n/a', 'platinum'], axis=1)
# merge with aggregated data
renewal_final = pd.merge(renewal_agg, most_common_values_dum, on=['CASE'], how='inner')

In [ ]:
# importing the aqua json file
aqua_df = pd.read_pickle("c:/Users/H008906/OneDrive - Principal Financial Group/expected value/qdf.pkl")
aqua_df = aqua_df[aqua_df['stat_cd'] == 4]

In [ ]:
# selecting columns from aqua_df
cols_to_use = ['contract', 'tot_lvs_cnt', 'sic_cd']

cols_not_use = ['grp_rprst_nmbr', 'grp_busn_id', 'vrsn_seq_nmbr', 'acq_prpsl_id', 'acq_vrsn_nmbr', 'st_cd', 'stat_cd', 'prfx_cd', 'grp_ofc_nmbr','mktr_nm', 'mktr_ofc_nm','medical',
                'rx', 'all_products','add','gul_life','reimbursemnt_arngmnt','health_savings_acct','aggregate_xcss_loss','vtl_add','wellness_program',
                'specific_xcss_loss', 'ltd','std vision','dental','supp_life','vtl_old','dep_life', 'life_add','vdp','supplife_add','vtl_add_old','child_vtl','spouse_vtl',
                'vtl','vol_critical_illness','accident', 'effective_dt','prem_amt']

aqua_df = aqua_df[cols_to_use]
aqua_df['contract']=aqua_df['contract'].str.strip()

In [ ]:
# merging the renewal data and aqua file
renewal_aqua = pd.merge(renewal_final, aqua_df, left_on='CASE', right_on='contract',how='inner')
renewal_aqua = renewal_aqua.drop('contract', axis = 1)
# get the current premium per life
renewal_aqua['premium_per_life'] = renewal_aqua['ENTERING_CURRENT_PREMIUM'] / renewal_aqua['tot_lvs_cnt']
renewal_aqua = renewal_aqua.drop(['ENTERING_CURRENT_PREMIUM', 'tot_lvs_cnt'], axis=1)

# counting cases per each sic code 
renewal_aqua['sic_count']  = renewal_aqua.groupby('sic_cd')['CASE'].transform('count')
# creating bins for each sic code group
bin_labels = ['sic_1', 'sic_2', 'sic_3']
renewal_aqua['sic_bin'] = pd.qcut(renewal_aqua['sic_count'], q=3, labels=bin_labels)
# dropping unused sic code columns
renewal_aqua = renewal_aqua.drop(['sic_cd', 'sic_count'], axis=1)

# get the dummies for each sic code and store it in a variable then change to 0 1
sic_dummies = pd.get_dummies(renewal_aqua.sic_bin)
sic_dummies = sic_dummies.astype(int)  
# concatenate the dummies to original dataframe
renewal_aqua_dum = pd.concat([renewal_aqua, sic_dummies], axis=1)
# drop values
renewal_aqua = renewal_aqua_dum.drop(['sic_bin'], axis=1)

In [ ]:
    def limits_function(data, column, min_obs, min_obs_sold, min_perc, max_perc):
        '''helper function for categorical_feature_limits function
            function to run for each categorical variable that is desired to have limits
            min_obs is a minimum number of observations required for category to be considered
            min_obs_sold is a minimum number of sales required for category to be considered
            min_perc, max_perc (in decimal format) are minimum and maximum close ratios on quotes for category to be considered
            all non-considered categories put into "Other" category'''

        #set column data type to category
        data[column] = data[column].astype('category')
        #category count
        items = data[column].value_counts().to_dict().items()
        #category sold count
        items_sold = data[data['cross_sell']==1][column].value_counts().to_dict().items()
        #category close ratio 
        items_perc = (data[data['cross_sell']==1].groupby([data[data['cross_sell']==1][column]])[column].count() / 
                    data.groupby([column, 'cross_sell'])[column].count().groupby(level=0).sum()).to_dict().items()
        #add Other category for column as possible category
        data[column]=data[column].cat.add_categories(['Other'])
        #if category does not meet criteria, reset to Other category
        data.loc[(data[column].isin([key for key, val in items if val < min_obs]))|
                (data[column].isin([key for key, val_sold in items_sold if val_sold < min_obs_sold]))|
                (data[column].isin([key for key, val in items_perc if not min_perc < val < max_perc])), column] = 'Other'
        data[column] = data[column].fillna('Other')
        #for categories that were moved to Other, delete
        data[column]=data[column].cat.remove_unused_categories()

        return data[column]


    def categorical_feature_limits(data):
        '''for the categorical features, some categories have very sparse examples - 
            limit use of categories to only those with an acceptable amount of examples
            uses limits_function to perform the limits'''

        data['sic_cd'] = limits_function(data, 'sic_cd', 5000, 100, 0.01, 0.4)

        return data

### Data Analysis

In [ ]:
# converting columns into categories 
def encode(data):
        columns=['cross_sell', 'DENTAL', 'LIFE', 'LTD', 'STD', 'PRODUCTSCOUNT','RG_binary', 'silver/elite', 'sic_1', 'sic_2', 'sic_3']
        for cat in columns:
            data[cat] = data[cat].astype('category')

        return data
        
model_data = encode(renewal_aqua)

In [ ]:
# categorical variables correlation matrix
# dropping variables not used in modeling
data_nominal = model_data.drop(['CASE'], axis=1)
# identifying nomial columns 
cat =identify_nominal_columns(data_nominal)
selected_column= data_nominal[cat]
# plotting heat map
categorical_correlation= associations(selected_column, filename= 'categorical_correlation.png', figsize=(10,10))

In [ ]:
# nominal variables correlation matrix 
data_ordinal = model_data.drop(['CASE', 'cross_sell', 'DENTAL', 'LIFE', 'LTD', 'STD', 'PRODUCTSCOUNT', 'RG_binary', 'silver/elite',  'sic_1', 'sic_2', 'sic_3'], axis=1)
# plotting correlation heatmap
dataplot = sns.heatmap(data_ordinal.corr(), cmap="YlGnBu", annot=True)
# displaying heatmap
plt.show()

In [ ]:
# the independent ordinal variables set
X = data_ordinal
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

In [ ]:
# resursive feature elimination 
def rfecv(self):
        SEED=42
        data=self
        X=data.drop(['cross_sell', 'CASE'], axis=1)
        y=data['cross_sell']
        ordinal_encoder=OrdinalEncoder()
        for col in X.select_dtypes(include='object'):
            X[col]=ordinal_encoder.fit_transform(X[col])
        model=RandomForestClassifier(random_state=SEED)
        rfecv=RFECV(estimator=model, cv=5)
        X_rfe=rfecv.fit_transform(X,y)
        selected_features=X.columns[rfecv.support_].to_list()
        print('Selected Features:')
        print(selected_features)

In [ ]:
#rfecv(model_data)

### Split Data

In [ ]:
# define folds and seeds
folds = 5 
seed=0
kf = KFold(n_splits = folds, shuffle = True, random_state=seed)

# target variable 
model_data['cross_sell'] = model_data['cross_sell'].astype('bool')

# split data into train and test
train, test = np.split(model_data.sample(frac=1, random_state=seed), [int(.8*len(model_data))])


# define features and target 
y_train = train['cross_sell']
X_train = train.drop(['cross_sell', 'CASE'], axis=1)
X_train_index = train['CASE']
y_test = test['cross_sell']
X_test = test.drop(['cross_sell', 'CASE'], axis=1)
X_test_index = test['CASE']

## Data Modeling

### Lgb model

In [ ]:
# define the category columns 
catCols = [i for i,v in enumerate(X_train.dtypes) if str(v)=='category']

In [ ]:
# lgbm model
for train_idx, val_idx in kf.split(X_train, y_train):
    train_x, train_y = X_train.iloc[train_idx],y_train.iloc[train_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    lgb_train = lgb.Dataset(train_x, label=train_y)
    lgb_val = lgb.Dataset(val_x, label=val_y, reference=lgb_train)
    params_lgb = {
        'boosting_type': 'dart',
        'metric': ['auc','f1','binary_error'],
        'objective':'binary',
        'bagging_freq': 0,
        'num_boost_round': 500,
        'verbose': 0,
        'is_unbalance': True,
        'path_smooth':0.5,
        'num_terations':500,
        'early_stopping_round':100,
        'extra_trees':True,
        'bagging_fraction': 0.8,
        'feature_fraction': 0.4,
        'learning_rate': 0.9,
        'max_bin': 80,
        'max_depth': 25,
        'min_data_in_leaf': 70,
        'min_sum_hessian_in_leaf': 20,
        'num_leaves': 40,
        'subsample': 0.25
        }
    lgbm = lgb.train(params_lgb,lgb_train, valid_sets=lgb_val, categorical_feature=catCols)
    feature_imp = pd.DataFrame(sorted(zip(lgbm.feature_importance(),X_train.columns)), columns=['Value','Feature'])


In [ ]:
# plotting lgbm
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

In [ ]:
# classification report for lgbm model 
model_test = lgbm.predict(X_test)
model_train = lgbm.predict(X_train)
ypred = [int(p>=0.6) for p in model_test]
print(classification_report(y_test, ypred))

In [ ]:
# find the optimal parameters 
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=3, random_seed=6,n_estimators=10000, 
                            output_process=False):
    '''function for using bayesian optimization to determine optimal training parameters for lightgbm model'''
    
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, min_data_in_leaf,min_sum_hessian_in_leaf,subsample):
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)
        
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, metrics=['auc'])
        #print(cv_result)       
        return max(cv_result['valid auc-mean'])
     
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.01, 1.0),
                                            'num_leaves': (24, 80),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 30),
                                            'max_bin':(20,90),
                                            'min_data_in_leaf': (20, 80),
                                            'min_sum_hessian_in_leaf':(0,100),
                                           'subsample': (0.01, 1.0)}, random_state=200)

    
    #n_iter: How many steps of bayesian optimization you want to perform. 
    #init_points: How many steps of random exploration you want to perform.
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    mae=[]
    for model in range(len( lgbBO.res)):
        mae.append(lgbBO.res[model]['target'])
    
    # return best parameters
    return lgbBO.res[pd.Series(mae).idxmin()]['target'],lgbBO.res[pd.Series(mae).idxmin()]['params']

opt_params = bayes_parameter_opt_lgb(X_train, y_train, 
                                     init_round=5, opt_round=10, n_folds=folds, random_seed=seed ,n_estimators=10000)

In [ ]:
# create dictionary of optimal parameters 
opt_params[1]["num_leaves"] = int(round(opt_params[1]["num_leaves"]))
opt_params[1]['max_depth'] = int(round(opt_params[1]['max_depth']))
opt_params[1]['min_data_in_leaf'] = int(round(opt_params[1]['min_data_in_leaf']))
opt_params[1]['max_bin'] = int(round(opt_params[1]['max_bin']))
opt_params[1]['boosting_type']= 'dart'
opt_params[1]['objective']='regression'
opt_params[1]['metric']= 'mae'
opt_params[1]['is_unbalance']=True
opt_params[1]['num_class']=y_train.nunique()+1
opt_params=opt_params[1]